# 5: Logistic classification

## import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.__version__

'1.1.0'

In [3]:
torch.manual_seed(1)

In [4]:
import sys
import torch_custom_functions as tcf

In [5]:
sys.version_info

sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)

## Low level setting

data setting

In [6]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [7]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

weight, bias setting

In [8]:
w = torch.zeros((2, 1), requires_grad = True)
b = torch.zeros((1), requires_grad = True)

## Computing the Hypothesis

hypothesis

## Computing the Hypothesis

$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

PyTorch has a `torch.exp()` function that resembles the exponential function.

In [9]:
h = 1 / (1 + torch.exp(-x_train.matmul(w) - b))

In [10]:
h

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)

In [11]:
torch.sigmoid((x_train.matmul(w) + b))

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)

loss setting

## Computing the Cost Function (Low-level)

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

We want to measure the difference between `hypothesis` and `y_train`.

In [12]:
loss = tcf.ce(h, y_train)
loss

tensor(0.6931, grad_fn=<NegBackward>)

optimizer settings

In [13]:
optimizer = optim.SGD([w, b], lr = 0.01)

In [14]:
w = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w, b], lr=1)


epoch = 1000
for i in range(epoch+1):
    # hypothesis
    h = torch.sigmoid(x_train.matmul(w)+b)
    
    # loss 
    loss = tcf.ce(h, y_train)
    
    # gradient descent & update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if i % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            i, epoch, loss.item()
        ))
    
    

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


# High level setting

In [15]:
class LogisticClassfiicationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        return self.sigmoid(self.linear(x))

In [16]:
logistic_model = LogisticClassfiicationModel()

In [17]:
logistic_model

LogisticClassfiicationModel(
  (linear): Linear(in_features=2, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [18]:
list(logistic_model.parameters())

[Parameter containing:
 tensor([[ 0.3643, -0.3121]], requires_grad=True),
 Parameter containing:
 tensor([-0.1371], requires_grad=True)]

In [19]:
print(list(logistic_model.parameters())[0])      

Parameter containing:
tensor([[ 0.3643, -0.3121]], requires_grad=True)


In [20]:
x_train.shape

torch.Size([6, 2])

In [21]:
logistic_model = LogisticClassfiicationModel()
optimizer = optim.SGD(logistic_model.parameters(), lr = 1)

epoch = 1000
for i in range(epoch+1):
    # hypothesis
    h = logistic_model(x_train)
    
    # loss
    loss = F.binary_cross_entropy(h, y_train)
    
    # gradient descent & update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if i % 100 == 0:
        params = list(logistic_model.parameters())
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            i, epoch, loss.item()
        ))

Epoch    0/1000 Cost: 0.614994
Epoch  100/1000 Cost: 0.140280
Epoch  200/1000 Cost: 0.082549
Epoch  300/1000 Cost: 0.058858
Epoch  400/1000 Cost: 0.045878
Epoch  500/1000 Cost: 0.037649
Epoch  600/1000 Cost: 0.031951
Epoch  700/1000 Cost: 0.027766
Epoch  800/1000 Cost: 0.024558
Epoch  900/1000 Cost: 0.022020
Epoch 1000/1000 Cost: 0.019960
